In [ ]:
!pip install earthengine-api --quiet > /dev/null 2>&1
!pip install earthengine-api geemap folium -q --quiet > /dev/null 2>&1

In [ ]:
import folium
from geemap import Map
import ee

# 1. Google Earth Engine の認証
ee.Authenticate()
# 'your-project-id' を実際の Google Cloud プロジェクト ID に置き換えてください
ee.Initialize(project='coral-mode-426910-u5')

# 2. 気温データ（ERA5-Land, 2020年7月）
dataset = (ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_BY_HOUR')
           .filter(ee.Filter.date('2020-07-01', '2020-08-01'))
           .select('temperature_2m'))

# 3. 可視化の設定（暖色系ヒートマップ）
visualization = {
    'min': 273.0,  # 0°C
    'max': 313.0,  # 40°C
    'palette': [
        'blue',      # 低温（寒い）
        'green',     # 温暖
        'yellow',    # 暖かい
        'red',       # 高温（暑い）
        'darkred'    # 非常に暑い
    ]
}

# 4. geemap で地図を表示
# 地図の初期化（中心座標を設定）
m = Map(center=[20, 0], zoom=2)

# GEE のデータを地図に追加
m.addLayer(dataset.mean(), visualization, 'Average Temperature (°K)')

# folium で地図を描画
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [ ]:
import folium
from geemap import Map

# 1. Google Earth Engine の認証
import ee
ee.Authenticate()
# 'your-project-id' を実際の Google Cloud プロジェクト ID に置き換えてください
ee.Initialize(project='coral-mode-426910-u5')

# 2. 気温データ（ERA5-Land, 2020年7月）
dataset = (ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_BY_HOUR')
           .filter(ee.Filter.date('2020-07-01', '2020-08-01'))
           .select('temperature_2m'))

# 3. 日ごとの最大・最小温度を計算
daily_max = dataset.max()
daily_min = dataset.min()

# 4. 月間の温度変動（最大 - 最小）の計算
temp_variation = daily_max.subtract(daily_min)

# 5. 可視化の設定（暖色系ヒートマップ）
visualization = {
    'min': 5.0,  # 5K以上の温度変化
    'max': 30.0, # 30Kの変動があるエリアを強調
    'palette': [
        'fff5b1',  # 薄い黄色（5K）
        'ffdb5c',  # 明るい黄色（10K）
        'ffa500',  # オレンジ（15K）
        'ff7300',  # 赤オレンジ（20K）
        'ff3300',  # 赤（25K）
        'b20000'   # 深紅（30K）
    ]
}

# 6. geemap で地図を表示
# 地図の初期化（中心座標を設定）
m = Map(center=[20, 0], zoom=2)

# GEE のデータを地図に追加
m.addLayer(temp_variation, visualization, 'Temperature Variation (K)')

# folium で地図を描画
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…